<a href="https://colab.research.google.com/github/minxuanjun/stable-diffusion-webui-colab/blob/main/lite/stable_diffusion_inpainting_webui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget https://github.com/camenduru/gperftools/releases/download/v1.0/libtcmalloc_minimal.so.4 -O /content/libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20 triton==2.0.0 gradio_client==0.2.7 -U

!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui
!git clone https://huggingface.co/embed/negative /content/stable-diffusion-webui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/stable-diffusion-webui/models/Lora/positive
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/stable-diffusion-webui/models/ESRGAN -o 4x-UltraSharp.pth
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O /content/stable-diffusion-webui/scripts/run_n_times.py
!git clone -b v2.4 https://github.com/camenduru/deforum-for-automatic1111-webui /content/stable-diffusion-webui/extensions/deforum-for-automatic1111-webui
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!git clone -b v2.4 https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
!git clone -b v2.4 https://github.com/camenduru/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
!git clone -b v2.4 https://github.com/camenduru/sd-webui-tunnels /content/stable-diffusion-webui/extensions/sd-webui-tunnels
!git clone -b v2.4 https://github.com/camenduru/batchlinks-webui /content/stable-diffusion-webui/extensions/batchlinks-webui
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-catppuccin /content/stable-diffusion-webui/extensions/stable-diffusion-webui-catppuccin
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-rembg /content/stable-diffusion-webui/extensions/stable-diffusion-webui-rembg
!git clone -b v2.4 https://github.com/camenduru/stable-diffusion-webui-two-shot /content/stable-diffusion-webui/extensions/stable-diffusion-webui-two-shot
!git clone -b v2.4 https://github.com/camenduru/sd-webui-aspect-ratio-helper /content/stable-diffusion-webui/extensions/sd-webui-aspect-ratio-helper
!git clone -b v2.4 https://github.com/camenduru/asymmetric-tiling-sd-webui /content/stable-diffusion-webui/extensions/asymmetric-tiling-sd-webui
%cd /content/stable-diffusion-webui
!git reset --hard
!git -C /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai reset --hard

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/i15/resolve/main/sd-v1-5-inpainting.ckpt -d /content/stable-diffusion-webui/models/Stable-diffusion -o sd-v1-5-inpainting.ckpt

!sed -i -e '''/from modules import launch_utils/a\import os''' /content/stable-diffusion-webui/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/stable-diffusion-webui/modules/shared.py

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

In [ ]:
import torch
import diffusers

import torch
from diffusers import StableDiffusionPipeline
base_model = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(base_model, torch_dtype=torch.float16)

In [ ]:
print(pipe.components)
print(type(pipe.components))


In [ ]:
pipe = pipe.to("cuda")

## CLip tokenizer [Text Encoder tutorail](https://zhuanlan.zhihu.com/p/680103276)

CLIPTokenizer 会将文本拆分成各个单词，然后使用查表法将每个子单词转换为数字。
其中每个prompt 编码的最大长度是77。每个编码出的序列都有一些固定的token. *start_token*, *end_token*.
```python
'tokenizer': CLIPTokenizer(name_or_path='/root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/f03de327dd89b501a01da37fc5240cf4fdba85a1/tokenizer', vocab_size=49408, model_max_length=77, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'
    }, clean_up_tokenization_spaces=False),  added_tokens_decoder={
        49406: AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
        49407: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
    }
```

In [ ]:
tokenizer = pipe.components["tokenizer"]
print(type(tokenizer))
prompt = ["a dog wearning hat"]
prompt = ['tokenizer = pipe.components["tokenizer"]']
text_token = tokenizer(prompt, padding="max_length", max_length = tokenizer.model_max_length, truncation=True, return_tensors = "pt")
text_token = text_token.to("cpu")
print(text_token.input_ids.shape)
print(text_token)

for token in list(text_token.input_ids[0,:7]):
  print(f"{token}:{tokenizer.convert_ids_to_tokens(int(token.item()))}")

## Text Encoder
Text Encoder 就是将将tokenizer 生成的input_ids 转化为一个文本嵌入的特征，文本嵌入的特征维度是768。Text Encoder 就是， 输出特征维度是[77,768]

```python
'text_encoder': CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408,
    768)
      (position_embedding): Embedding(77,
    768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
)
```

In [ ]:
text_encoder = pipe.components["text_encoder"]
# get model structure
print(text_encoder)

with torch.no_grad():
  text_embed = text_encoder(text_token.input_ids.to("cuda"))
# print(text_embed)
print(f"hidden {text_embed.last_hidden_state.shape}  {text_embed.pooler_output.shape}")


我们可以通过控制  `num_inference_steps` 来控制去噪次数，去噪次数越多，结果越好。 默认值是50.

In [ ]:
prompt = "a photograph of an astronaut riding a horse"
%debug image = pipe(prompt, num_inference_steps=15).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

# Now to display an image you can either save it such as:
image.save(f"astronaut_rides_horse.png")

# or if you're in a google colab you can directly display it with
image